In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

# NLTK durdurma kelimelerini indir
nltk.download('stopwords')

# Stop words listesi (Türkçe)
stop_words = set(stopwords.words('turkish'))

# Metin Temizleme Fonksiyonu
def metin_temizle(metin):
    metin = metin.lower()
    metin = re.sub(r'[^a-zçğıöşü\s]', '', metin)
    metin = re.sub(r'\s+', ' ', metin).strip()
    metin = ' '.join([kelime for kelime in metin.split() if kelime not in stop_words])

    return metin

# Özellik Çıkarma Fonksiyonu (TF-IDF)
def ozellik_cikarma(metinler):
    vectorizer = TfidfVectorizer(min_df=1)
    tfidf_ozellikler = vectorizer.fit_transform(metinler)
    return tfidf_ozellikler


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def birads_dosyalari_isle(dizin_yollari):
    metinler = []
    etiketler = []

    for dizin_yolu in dizin_yollari:
        for dosya_adi in os.listdir(dizin_yolu):
            dosya_yolu = os.path.join(dizin_yolu, dosya_adi)
            if os.path.isfile(dosya_yolu):
                with open(dosya_yolu, 'r', encoding='utf-8') as file:
                    metin = file.read()
                    metin_temiz = metin_temizle(metin)
                    birads_etiketi = birads_etiket_belirle(metin)
                    if birads_etiketi is not None:
                        metinler.append(metin_temiz)
                        etiketler.append(birads_etiketi)

    return metinler, etiketler


In [ ]:
def ner_dosya_isle(dosya_yolu):
    veriler = jsonl_dosya_oku(dosya_yolu)
    metinler, etiketler = metin_ve_etiket_ayikla(veriler)
    temiz_metinler = [metin_temizle(metin) for metin in metinler]
    return temiz_metinler, etiketler


In [ ]:
import os
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# NLTK durdurma kelimelerini indir
nltk.download('stopwords')

# Stop words listesi (Türkçe)
stop_words = set(stopwords.words('turkish'))

# Metin Temizleme Fonksiyonu
def metin_temizle(metin):
    metin = metin.lower()
    metin = re.sub(r'[^a-zçğıöşü\s]', '', metin)
    metin = re.sub(r'\s+', ' ', metin).strip()
    metin = ' '.join([kelime for kelime in metin.split() if kelime not in stop_words])
    return metin

# BIRADS Etiket Hazırlığı Fonksiyonu
def birads_etiket_belirle(metin):
    metin = metin.lower()
    pattern = r"birads\s*[:\-]?\s*(\d)"
    match = re.search(pattern, metin)
    if match:
        return int(match.group(1))
    return None

# Özellik Çıkarma Fonksiyonu (TF-IDF)
def ozellik_cikarma(metinler):
    vectorizer = TfidfVectorizer(min_df=1)
    tfidf_ozellikler = vectorizer.fit_transform(metinler)
    return tfidf_ozellikler

# Dosya Okuma Fonksiyonu
def dosyalari_oku(dosya_yolu):
    try:
        with open(dosya_yolu, 'r', encoding='utf-8') as file:
            raporlar = file.read()
    except UnicodeDecodeError:
        with open(dosya_yolu, 'r', encoding='ISO-8859-1') as file:
            raporlar = file.read()
    return raporlar

# BIRADS Dosyalarını İşleme Fonksiyonu
def birads_dosyalari_isle(dizin_yollari):
    metinler = []
    etiketler = []

    for dizin_yolu in dizin_yollari:
        for dosya_adi in os.listdir(dizin_yolu):
            dosya_yolu = os.path.join(dizin_yolu, dosya_adi)
            if os.path.isfile(dosya_yolu):
                with open(dosya_yolu, 'r', encoding='utf-8') as file:
                    metin = file.read()
                    metin_temiz = metin_temizle(metin)
                    birads_etiketi = birads_etiket_belirle(metin)
                    if birads_etiketi is not None:
                        metinler.append(metin_temiz)
                        etiketler.append(birads_etiketi)

    return metinler, etiketler

# JSONL Dosyasını Okuma ve İşleme Fonksiyonu
def jsonl_dosyalarini_isle(dosya_yollari):
    metinler = []
    etiketler = []

    for dosya_yolu in dosya_yollari:
        with open(dosya_yolu, 'r', encoding='utf-8') as f:
            for satir in f:
                veri = json.loads(satir)
                metin = metin_temizle(veri.get('text', ''))
                etiket = veri.get('label', [])
                metinler.append(metin)
                etiketler.append(etiket)

    return metinler, etiketler

# BIRADS ve NER Veri Setlerini İşleyin ve Özellikleri Çıkarın
birads_klasor_yollari = [
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS1",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS2",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS3",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS4",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS5"
]

jsonl_dosya_yollari = [
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/1/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/2/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/3/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/4/all.jsonl"
]

# BIRADS Veri Seti İşle
birads_metinler, birads_etiketler = birads_dosyalari_isle(birads_klasor_yollari)
birads_ozellikler = ozellik_cikarma(birads_metinler)

# NER Veri Seti İşle
ner_metinler, ner_etiketler = jsonl_dosyalarini_isle(jsonl_dosya_yollari)
ner_ozellikler = ozellik_cikarma(ner_metinler)

# BIRADS Veri Seti için Model Eğitimi ve Değerlendirme
X_train, X_test, y_train, y_test = train_test_split(birads_ozellikler, birads_etiketler, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("BIRADS Accuracy:", accuracy_score(y_test, y_pred))
print("BIRADS Classification Report:\n", classification_report(y_test, y_pred))

# NER Veri Seti için Model Eğitimi ve Değerlendirme
# Bu kısım, etiketi NER etiketleme için uygun olacak şekilde özelleştirilmelidir.
# Burada bir örnek modelleme veriyoruz; ancak NER için spesifik bir model ve değerlendirme gerekecektir.
# Özellikle çoklu etiket (multi-label) sınıflandırma için uygun yöntemler kullanılmalıdır.

# Örnek bir çoklu etiket sınıflandırma modeli ve değerlendirmesi (NER etiketi formatına göre uyarlanmalı)
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_ner_train = mlb.fit_transform(ner_etiketler)
y_ner_test = mlb.transform(ner_etiketler)

X_train, X_test, y_train, y_test = train_test_split(ner_ozellikler, y_ner_train, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("NER Accuracy:", accuracy_score(y_test, y_pred))
print("NER Classification Report:\n", classification_report(y_test, y_pred, target_names=mlb.classes_))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


BIRADS Accuracy: 0.8618307426597582
BIRADS Classification Report:
               precision    recall  f1-score   support

           1       0.99      0.89      0.94        75
           2       0.91      0.62      0.74       103
           3       0.84      0.87      0.86       124
           4       0.85      0.90      0.87       133
           5       0.82      0.97      0.89       144

    accuracy                           0.86       579
   macro avg       0.88      0.85      0.86       579
weighted avg       0.87      0.86      0.86       579



TypeError: unhashable type: 'list'

In [ ]:
import os
import re
import json
import nltk
from nltk.corpus import stopwords

# NLTK durdurma kelimelerini indir
nltk.download('stopwords')

# Stop words listesi (Türkçe)
stop_words = set(stopwords.words('turkish'))

# Metin Temizleme Fonksiyonu
def metin_temizle(metin):
    metin = metin.lower()
    metin = re.sub(r'[^a-zçğıöşü\s]', '', metin)
    metin = re.sub(r'\s+', ' ', metin).strip()
    metin = ' '.join([kelime for kelime in metin.split() if kelime not in stop_words])
    return metin

# BIRADS Dosyalarını Okuma ve İşleme
def birads_dosyalari_isle(dizin_yollari):
    metinler = []
    etiketler = []

    for dizin_yolu in dizin_yollari:
        for dosya_adi in os.listdir(dizin_yolu):
            dosya_yolu = os.path.join(dizin_yolu, dosya_adi)
            if os.path.isfile(dosya_yolu):
                with open(dosya_yolu, 'r', encoding='utf-8') as file:
                    metin = file.read()
                    metin_temiz = metin_temizle(metin)
                    birads_etiketi = birads_etiket_belirle(metin)
                    if birads_etiketi is not None:
                        metinler.append(metin_temiz)
                        etiketler.append(birads_etiketi)

    return metinler, etiketler

# JSONL Dosyalarını Okuma ve İşleme
def jsonl_dosyalarini_isle(dosya_yollari):
    metinler = []
    etiketler = []

    for dosya_yolu in dosya_yollari:
        with open(dosya_yolu, 'r', encoding='utf-8') as f:
            for satir in f:
                veri = json.loads(satir)
                metin = metin_temizle(veri.get('text', ''))
                etiket = veri.get('label', [])
                if not isinstance(etiket, list):
                    etiket = [etiket]
                metinler.append(metin)
                etiketler.append(etiket)

    return metinler, etiketler

# BIRADS ve NER veri yolları
birads_klasor_yollari = [
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS1",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS2",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS3",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS4",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS5"
]

jsonl_dosya_yollari = [
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/1/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/2/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/3/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/4/all.jsonl"
]

# BIRADS ve NER Veri Setlerini İşle
birads_metinler, birads_etiketler = birads_dosyalari_isle(birads_klasor_yollari)
ner_metinler, ner_etiketler = jsonl_dosyalarini_isle(jsonl_dosya_yollari)

# Veri Kontrolü
print("BIRADS Metin Sayısı:", len(birads_metinler))
print("BIRADS Etiket Sayısı:", len(birads_etiketler))
print("NER Metin Sayısı:", len(ner_metinler))
print("NER Etiket Sayısı:", len(ner_etiketler))
print("Örnek BIRADS Etiket:", birads_etiketler[:5])
print("Örnek NER Etiket:", ner_etiketler[:5])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


BIRADS Metin Sayısı: 2892
BIRADS Etiket Sayısı: 2892
NER Metin Sayısı: 1028
NER Etiket Sayısı: 1028
Örnek BIRADS Etiket: [1, 1, 1, 1, 1]
Örnek NER Etiket: [[[32, 47, 'ANAT'], [48, 59, 'ANAT'], [60, 66, 'OBS-PRESENT'], [96, 106, 'OBS-ABSENT'], [110, 121, 'OBS-ABSENT'], [139, 151, 'ANAT'], [152, 165, 'OBS-PRESENT'], [165, 173, 'OBS-PRESENT'], [190, 202, 'ANAT'], [203, 216, 'OBS-PRESENT'], [217, 226, 'OBS-PRESENT'], [228, 238, 'ANAT'], [239, 255, 'ANAT'], [255, 273, 'ANAT'], [273, 287, 'OBS-PRESENT'], [287, 303, 'OBS-PRESENT'], [313, 330, 'ANAT'], [331, 350, 'OBS-ABSENT'], [375, 383, 'IMPRESSION']], [[36, 50, 'ANAT'], [51, 65, 'ANAT'], [66, 75, 'OBS-PRESENT'], [77, 89, 'ANAT'], [90, 95, 'OBS-PRESENT'], [97, 108, 'OBS-PRESENT'], [110, 124, 'ANAT'], [125, 155, 'OBS-ABSENT'], [157, 180, 'OBS-ABSENT'], [186, 205, 'OBS-ABSENT'], [245, 268, 'ANAT'], [270, 290, 'OBS-PRESENT'], [312, 320, 'IMPRESSION']], [[31, 45, 'ANAT'], [46, 60, 'ANAT'], [61, 71, 'OBS-PRESENT'], [72, 84, 'ANAT'], [85, 99, 'ANA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Özellik Çıkarma Fonksiyonu (TF-IDF)
def ozellik_cikarma(metinler):
    vectorizer = TfidfVectorizer(min_df=1)
    tfidf_ozellikler = vectorizer.fit_transform(metinler)
    return tfidf_ozellikler

# Özellikleri Çıkar
birads_ozellikler = ozellik_cikarma(birads_metinler)
ner_ozellikler = ozellik_cikarma(ner_metinler)

# Özellik Boyut Kontrolü
print("BIRADS TF-IDF Özellik Boyutu:", birads_ozellikler.shape)
print("NER TF-IDF Özellik Boyutu:", ner_ozellikler.shape)


BIRADS TF-IDF Özellik Boyutu: (2892, 2167)
NER TF-IDF Özellik Boyutu: (1028, 1806)


In [ ]:
import os
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

# NLTK durdurma kelimelerini indir
nltk.download('stopwords')

# Stop words listesi (Türkçe)
stop_words = set(stopwords.words('turkish'))

# Metin Temizleme Fonksiyonu
def metin_temizle(metin):
    metin = metin.lower()
    metin = re.sub(r'[^a-zçğıöşü\s]', '', metin)
    metin = re.sub(r'\s+', ' ', metin).strip()
    metin = ' '.join([kelime for kelime in metin.split() if kelime not in stop_words])
    return metin

# BIRADS Etiket Hazırlığı Fonksiyonu
def birads_etiket_belirle(metin):
    metin = metin.lower()
    pattern = r"birads\s*[:\-]?\s*(\d)"
    match = re.search(pattern, metin)
    if match:
        return int(match.group(1))
    return None

# Özellik Çıkarma Fonksiyonu (TF-IDF)
def ozellik_cikarma(metinler):
    vectorizer = TfidfVectorizer(min_df=1)
    tfidf_ozellikler = vectorizer.fit_transform(metinler)
    return tfidf_ozellikler

# Dosya Okuma Fonksiyonu
def dosyalari_oku(dosya_yolu):
    try:
        with open(dosya_yolu, 'r', encoding='utf-8') as file:
            raporlar = file.read()
    except UnicodeDecodeError:
        with open(dosya_yolu, 'r', encoding='ISO-8859-1') as file:
            raporlar = file.read()
    return raporlar

# BIRADS Dosyalarını İşleme Fonksiyonu
def birads_dosyalari_isle(dizin_yollari):
    metinler = []
    etiketler = []

    for dizin_yolu in dizin_yollari:
        for dosya_adi in os.listdir(dizin_yolu):
            dosya_yolu = os.path.join(dizin_yolu, dosya_adi)
            if os.path.isfile(dosya_yolu):
                with open(dosya_yolu, 'r', encoding='utf-8') as file:
                    metin = file.read()
                    metin_temiz = metin_temizle(metin)
                    birads_etiketi = birads_etiket_belirle(metin)
                    if birads_etiketi is not None:
                        metinler.append(metin_temiz)
                        etiketler.append(birads_etiketi)

    return metinler, etiketler

# JSONL Dosyasını Okuma ve İşleme Fonksiyonu
def jsonl_dosyalarini_isle(dosya_yollari):
    metinler = []
    etiketler = []

    for dosya_yolu in dosya_yollari:
        with open(dosya_yolu, 'r', encoding='utf-8') as f:
            for satir in f:
                veri = json.loads(satir)
                metin = metin_temizle(veri.get('text', ''))
                etiket = veri.get('label', [])
                # Etiketin liste olmasını sağla
                if not isinstance(etiket, list):
                    etiket = [etiket]
                metinler.append(metin)
                etiketler.append(etiket)

    # Etiketlerin düzgün formatta olup olmadığını kontrol et
    etiketler = [[etiket] if not isinstance(etiket, list) else etiket for etiket in etiketler]

    return metinler, etiketler

# BIRADS ve NER Veri Setlerini İşleyin ve Özellikleri Çıkarın
birads_klasor_yollari = [
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS1",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS2",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS3",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS4",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS5"
]

jsonl_dosya_yollari = [
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/1/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/2/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/3/all.jsonl",
    "/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/4/all.jsonl"
]

# BIRADS Veri Seti İşle
birads_metinler, birads_etiketler = birads_dosyalari_isle(birads_klasor_yollari)
birads_ozellikler = ozellik_cikarma(birads_metinler)

# NER Veri Seti İşle
ner_metinler, ner_etiketler = jsonl_dosyalarini_isle(jsonl_dosya_yollari)

# Etiketlerin düzgün biçimlendirilip biçimlendirilmediğini kontrol etme
print("İlk 5 NER Etiketi:", ner_etiketler[:5])

# Multi-label binarizer kullanarak etiketleri dönüştürme
mlb = MultiLabelBinarizer()

# İç içe liste durumunu çözmek için düzleştir ve stringe dönüştür
flattened_etiketler = []
for etiket in ner_etiketler:
    flattened_etiketler.append([str(item) for sublist in etiket for item in sublist])

y_train_ner_bin = mlb.fit_transform(flattened_etiketler)

# Etiket Boyut Kontrolü
print("Dönüştürülmüş NER Etiket Boyutu:", y_train_ner_bin.shape)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


İlk 5 NER Etiketi: [[[32, 47, 'ANAT'], [48, 59, 'ANAT'], [60, 66, 'OBS-PRESENT'], [96, 106, 'OBS-ABSENT'], [110, 121, 'OBS-ABSENT'], [139, 151, 'ANAT'], [152, 165, 'OBS-PRESENT'], [165, 173, 'OBS-PRESENT'], [190, 202, 'ANAT'], [203, 216, 'OBS-PRESENT'], [217, 226, 'OBS-PRESENT'], [228, 238, 'ANAT'], [239, 255, 'ANAT'], [255, 273, 'ANAT'], [273, 287, 'OBS-PRESENT'], [287, 303, 'OBS-PRESENT'], [313, 330, 'ANAT'], [331, 350, 'OBS-ABSENT'], [375, 383, 'IMPRESSION']], [[36, 50, 'ANAT'], [51, 65, 'ANAT'], [66, 75, 'OBS-PRESENT'], [77, 89, 'ANAT'], [90, 95, 'OBS-PRESENT'], [97, 108, 'OBS-PRESENT'], [110, 124, 'ANAT'], [125, 155, 'OBS-ABSENT'], [157, 180, 'OBS-ABSENT'], [186, 205, 'OBS-ABSENT'], [245, 268, 'ANAT'], [270, 290, 'OBS-PRESENT'], [312, 320, 'IMPRESSION']], [[31, 45, 'ANAT'], [46, 60, 'ANAT'], [61, 71, 'OBS-PRESENT'], [72, 84, 'ANAT'], [85, 99, 'ANAT'], [100, 106, 'OBS-PRESENT'], [107, 115, 'ANAT'], [116, 128, 'OBS-PRESENT'], [129, 134, 'OBS-PRESENT'], [135, 139, 'ANAT'], [143, 157,

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier

# BIRADS Veri Seti için Model Eğitimi ve Değerlendirme
X_train_birads, X_test_birads, y_train_birads, y_test_birads = train_test_split(birads_ozellikler, birads_etiketler, test_size=0.2, random_state=42)

model_birads = MultinomialNB()
model_birads.fit(X_train_birads, y_train_birads)

y_pred_birads = model_birads.predict(X_test_birads)
print("BIRADS Accuracy:", accuracy_score(y_test_birads, y_pred_birads))
print("BIRADS Classification Report:\n", classification_report(y_test_birads, y_pred_birads))

# NER Veri Seti için MultiLabelBinarizer Kullanarak Etiketleri Dönüştürme
mlb = MultiLabelBinarizer()

# Ner etiketlerini düzleştir ve string'e dönüştür
flattened_etiketler = []
for etiket in ner_etiketler:
    flattened_etiketler.append([str(item) for item in etiket])

# Etiketleri binarize et
y_ner_bin = mlb.fit_transform(flattened_etiketler)

# NER Veri Seti için Model Eğitimi ve Değerlendirme
X_train_ner, X_test_ner, y_train_ner, y_test_ner = train_test_split(ner_ozellikler, y_ner_bin, test_size=0.2, random_state=42)

# Çoklu Etiket Sınıflandırma için uygun bir model kullanın
model_ner = MultiOutputClassifier(MultinomialNB())
model_ner.fit(X_train_ner, y_train_ner)

y_pred_ner = model_ner.predict(X_test_ner)
print("NER Accuracy:", accuracy_score(y_test_ner, y_pred_ner))
print("NER Classification Report:\n", classification_report(y_test_ner, y_pred_ner, target_names=mlb.classes_))


In [ ]:
import json

# Dosya yolunu belirtin ve dosyayı satır satır okuyun
data = []
with open('/content/drive/MyDrive/Colab Notebooks/NLP-Teknofest24/NLPJSON/combined_all.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))  # Her satırı JSON olarak yükle ve listeye ekle

# Etiketli metinleri görselleştiren bir fonksiyon, IMPRESSION etiketlerini atlar ve "SONUÇ:" kısmını çıkarır
def visualize_labeled_text(entry):
    labeled_text = entry['text']  # Orijinal metin
    output_text = ""
    labels = entry['label']

    # "SONUÇ:" ile başlayan metni çıkartmak için kontrol et
    if "SONUÇ" in labeled_text:
        labeled_text = labeled_text.split("SONUÇ")[0]

    last_index = 0
    for start, end, label in labels:
        # IMPRESSION etiketlerini atla
        if label == "IMPRESSION":
            continue

        # Etiketlenmiş metin kısmını oluştur
        output_text += labeled_text[last_index:start]  # Etiket öncesi metin ekle
        output_text += f"{labeled_text[start:end]} [{label}]"  # Etiketli metin kısmını ekle
        last_index = end

    # Etiketlerden sonraki kalan metni ekle
    output_text += labeled_text[last_index:]

    return output_text

# Her bir örnek için etiketli metinleri görüntüleme
for entry in data:
    print(f"Metin ID: {entry['id']}")
    labeled_text = visualize_labeled_text(entry)
    print(labeled_text)
    print("\n" + "-"*50 + "\n")

Görüntülenen çıkış son 5000 satıra kısaltıldı.
--------------------------------------------------

Metin ID: 620
BİLATERAL MAMOGRAFİ İNCELEMESİ
Her iki meme [ANAT] tip C dansite [OBS-PRESENT] paternindedir [OBS-PRESENT].
Her iki meme [ANAT] cilt kalınlığı [ANAT] normal sınırlarda [OBS-PRESENT] areola [ANAT] tabiidir [OBS-PRESENT].
Her iki memede [ANAT] yapı distorsiyonu [OBS-ABSENT], patolojik mikrokalsifikasyon [OBS-ABSENT] yada spiküle konturlu [OBS-ABSENT] kitle lezyonu [OBS-ABSENT] saptanmamıştır.
Sol mem [ANAT] üst dış kadranda [ANAT] gruplaşmış [ANAT] punkrat [OBS-PRESENT] kalsifikasyon alanı [OBS-PRESENT] izlendi
Her iki aksiller bölgede [ANAT] 1cm yi geçmeyen [OBS-PRESENT] santrali [ANAT] lusent [OBS-PRESENT] lenf nodları [OBS-PRESENT] ile uyumlu opasiteler [OBS-PRESENT] izlenmiştir. 
 


--------------------------------------------------

Metin ID: 621
BİLATERAL MAMOGRAFİ İNCELEMESİ:
Mamografi cihazının tanısal olanakları dahilinde elde edilen; her iki memenin mediolateral obl